In [158]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100

## Download data

In [41]:
import sys
import os

sys.path.append(os.path.abspath('../..'))

In [10]:
data_root = '../../_data/kaggle/rossman'
!kaggle competitions download -c rossmann-store-sales -p $data_root

sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
store.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [34]:
import urllib.request
import os
urllib.request.urlretrieve('http://files.fast.ai/part2/lesson14/rossmann.tgz', os.path.join(data_root, 'rossmann.tgz'))

('../../_data/kaggle/rossman/rossmann.tgz',
 <http.client.HTTPMessage at 0x109515518>)

In [47]:
from glob import glob
import shutil
#for zipfile in glob(data_root + '/*.(zip)|(tgz)'):
for zipfile in glob(data_root + '/*.tgz'):
    shutil.unpack_archive(zipfile, data_root)

In [48]:
ls $data_root

googletrend.csv            store_states.csv
rossmann.tgz               test.csv
sample_submission.csv      test.csv.zip
sample_submission.csv.zip  train.csv
state_names.csv            train.csv.zip
store.csv                  weather.csv
store.csv.zip


## Preprocessing

In [42]:
import pandas as pd
from pandas_summary import DataFrameSummary
from IPython.display import display

In [63]:
table_names = [os.path.splitext(os.path.basename(f))[0] for f in glob(data_root + '/*.csv')]
tables = [pd.read_csv(f, low_memory=False) for f in glob(data_root + '/*.csv')]
print(', '.join(table_names))

googletrend, sample_submission, state_names, store, store_states, test, train, weather


In [98]:
for name, tb in zip(table_names, tables):
    print(name)
    display(tb.head())

googletrend


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


sample_submission


,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


state_names


...

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


train


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


weather


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [56]:
for tb in tables:
    display(DataFrameSummary(tb).summary())

,file,week,trend
count,NaN,NaN,2072
mean,NaN,NaN,63.8142
std,NaN,NaN,12.6502
min,NaN,NaN,0
25%,NaN,NaN,55
50%,NaN,NaN,64
75%,NaN,NaN,72
max,NaN,NaN,100
counts,2072,2072,2072
uniques,14,148,68


,Id,Sales
count,41088,41088
mean,20544.5,0
std,11861.2,0
min,1,0
25%,10272.8,0
50%,20544.5,0
75%,30816.2,0
max,41088,0
counts,41088,41088
uniques,41088,1


,StateName,State
count,16,16
unique,16,16
top,Hessen,BE
freq,1,1
counts,16,16
uniques,16,16
missing,0,0
missing_perc,0%,0%
types,unique,unique


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115,NaN,NaN,1112,761,761,1115,571,571,NaN
mean,558,NaN,NaN,5404.9,7.2247,2008.67,0.512108,23.5954,2011.76,NaN
std,322.017,NaN,NaN,7663.17,3.21235,6.19598,0.500078,14.142,1.67494,NaN
min,1,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,279.5,NaN,NaN,717.5,4,2006,0,13,2011,NaN
50%,558,NaN,NaN,2325,8,2010,1,22,2012,NaN
75%,836.5,NaN,NaN,6882.5,10,2013,1,37,2013,NaN
max,1115,NaN,NaN,75860,12,2015,1,50,2015,NaN
counts,1115,1115,1115,1112,761,761,1115,571,571,571
uniques,1115,4,3,654,12,23,2,24,7,3


,Store,State
count,1115,NaN
mean,558,NaN
std,322.017,NaN
min,1,NaN
25%,279.5,NaN
50%,558,NaN
75%,836.5,NaN
max,1115,NaN
counts,1115,1115
uniques,1115,12


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
count,41088,41088,41088,NaN,41077,41088,NaN,41088
mean,20544.5,555.9,3.97917,NaN,0.854322,0.395833,NaN,0.443487
std,11861.2,320.274,2.01548,NaN,0.352787,0.489035,NaN,0.496802
min,1,1,1,NaN,0,0,NaN,0
25%,10272.8,279.75,2,NaN,1,0,NaN,0
50%,20544.5,553.5,4,NaN,1,0,NaN,0
75%,30816.2,832.25,6,NaN,1,1,NaN,1
max,41088,1115,7,NaN,1,1,NaN,1
counts,41088,41088,41088,41088,41077,41088,41088,41088
uniques,41088,856,7,48,2,2,2,2


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056
min,1,1,NaN,0,0,0,0,NaN,0
25%,280,2,NaN,3727,405,1,0,NaN,0
50%,558,4,NaN,5744,609,1,0,NaN,0
75%,838,6,NaN,7856,837,1,1,NaN,0
max,1115,7,NaN,41551,7388,1,1,NaN,1
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209
uniques,1115,7,942,21734,4086,2,2,4,2


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
count,NaN,NaN,15840,15840,15840,15840,15840,15840,15840,15840,...,15459,15459,15459,15840,15840,3604,15840,14667,NaN,15840
mean,NaN,NaN,14.6441,10.389,6.19899,8.58782,6.20581,3.62614,93.6596,74.2829,...,24.0576,12.2398,7.02516,22.7666,11.9722,48.8643,0.831718,5.55131,NaN,175.897
std,NaN,NaN,8.64601,7.37926,6.52639,6.24478,6.08677,6.12839,7.67853,13.4866,...,8.9768,5.06794,4.9806,8.98862,5.87284,13.027,2.51351,1.68771,NaN,101.589
min,NaN,NaN,-11,-13,-15,-14,-15,-73,44,30,...,0,0,0,3,2,21,0,0,NaN,-1
25%,NaN,NaN,8,4,1,4,2,-1,90.75,65,...,14,10,3,16,8,39,0,5,NaN,80
50%,NaN,NaN,15,11,7,9,7,4,94,76,...,31,11,7,21,11,48,0,6,NaN,202
75%,NaN,NaN,21,16,11,13,11,8,100,85,...,31,14,10,27,14,55,0.25,7,NaN,256
max,NaN,NaN,39,31,24,25,20,19,100,100,...,31,31,31,101,53,111,58.93,8,NaN,360
counts,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,...,15459,15459,15459,15840,15840,3604,15840,14667,11889,15840
uniques,16,990,51,45,40,40,36,40,53,71,...,24,32,24,44,29,47,41,9,21,362


In [97]:
googletrend, sample_submission, state_names, store, store_states, test, train, weather = tables

In [78]:
len(test), len(train)

(41088, 1017209)

In [86]:
sorted(train.Date.unique())[-10:]

['2015-07-22',
 '2015-07-23',
 '2015-07-24',
 '2015-07-25',
 '2015-07-26',
 '2015-07-27',
 '2015-07-28',
 '2015-07-29',
 '2015-07-30',
 '2015-07-31']

In [87]:
sorted(test.Date.unique())[:10]

['2015-08-01',
 '2015-08-02',
 '2015-08-03',
 '2015-08-04',
 '2015-08-05',
 '2015-08-06',
 '2015-08-07',
 '2015-08-08',
 '2015-08-09',
 '2015-08-10']

In [115]:
googletrend.file.unique()

array(['Rossmann_DE_SN', 'Rossmann_DE', 'Rossmann_DE_BY',
       'Rossmann_DE_TH', 'Rossmann_DE_NW', 'Rossmann_DE_BE',
       'Rossmann_DE_RP', 'Rossmann_DE_BW', 'Rossmann_DE_NI',
       'Rossmann_DE_SH', 'Rossmann_DE_HE', 'Rossmann_DE_ST',
       'Rossmann_DE_HH', 'Rossmann_DE_SL'], dtype=object)

In [ ]:
# clean up time
# for name in ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']:
#     store[name] = store[name].astype(np.int32)

In [147]:
# clean up weath
weather['Date'] = pd.to_datetime(weather.Date)

In [130]:
# clean up trend
clean_trend = googletrend.copy()
clean_trend['Date'] = pd.to_datetime(googletrend.week.str.split(' - ', expand=True)[0])
clean_trend['State'] = googletrend.file.str.split('_', expand=True)[2]
de_trend = clean_trend[clean_trend.file == 'Rossmann_DE']
clean_trend = clean_trend.drop(columns=['week', 'file'])
de_trend = de_trend.drop(columns=['week', 'file', 'State'])
clean_trend.head()

,trend,Date,State
0,96,2012-12-02,SN
1,95,2012-12-09,SN
2,91,2012-12-16,SN
3,48,2012-12-23,SN
4,67,2012-12-30,SN


In [132]:
de_trend.head()

,trend,Date
148,77,2012-12-02
149,76,2012-12-09
150,85,2012-12-16
151,59,2012-12-23
152,61,2012-12-30


In [269]:
from datetime import datetime

def expend_date(df, expend=None):
    df = df.copy()
    df['Date'] = pd.to_datetime(df.Date)
    
    if not expend:
        expend = ['Year', 'Month', 'Day', 'Week']
    d = {'Year': df.Date.dt.year,
         'Month': df.Date.dt.month,
         'Day': df.Date.dt.day,
         'Week': df.Date.dt.week}
    for label in expend:
        df[label] = d[label]

    return df
    
def preprocess(data_set, joins, drops=[], nans=[]):
    d = expend_date(data_set)
    
    for tmp in joins:
        if len(tmp) == 3:
            right, left_on, right_on = tmp
        else:
            right, left_on = tmp
            right_on = left_on
        d = pd.merge(d, right, how='left', left_on=left_on, right_on=right_on, suffixes=("", "_r"))
    if drops:
        d = d.drop(columns=drops)
    
    for tmp in nans:
        if len(tmp) == 2:
            name, val = tmp
            d[name] = d[name].fillna(val)
        else:
            name, val, val_type = tmp
            d[name] = d[name].fillna(val).astype(val_type)
    # StatHolidy from ['0', 'a', 'b', 'c'] to [True, False]
    d.StateHoliday = data_set.StateHoliday == '0'
    
    # create new feature 'CompetitionOpenSince'
    d['CompetitionOpenSince'] = pd.to_datetime(
        d.apply(lambda x: datetime(x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 15), axis=1))
    d['CompetitionDaysOpen'] = (d.Date - d.CompetitionOpenSince).dt.days
    
    return d

merge_right = [
    (store, 'Store'),
    (store_states, 'Store'),
    (state_names, 'State'),
    (weather, ['StateName', 'Date'], ['file', 'Date']),
    (expend_date(clean_trend, ['Year', 'Week']), ['State', 'Year', 'Week']),
]
fill_nan = [
    ('CompetitionOpenSinceMonth', 1, np.int32),
    ('CompetitionOpenSinceYear', 1900, np.int32),
    ('Promo2SinceWeek', 1, np.int32),
    ('Promo2SinceYear', 1900, np.int32),
    ('CompetitionDistance', 0, np.int32),
    ('trend',0, np.int32),
    ('CloudCover', -1, np.int32),
#     ('PromoInterval'),
#     ('Max_VisibilityKm'),
#     ('Mean_VisibilityKm'),
#     ('Min_VisibilitykM'),
#     ('Max_Gust_SpeedKm_h',),
#     ('Events',),
]
train_processed = preprocess(train, merge_right, drops=['file', 'Date_r', 'StateName'],
                             nans=fill_nan)
train_processed.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,Week,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,trend,CompetitionOpenSince,CompetitionDaysOpen
0,1,5,2015-07-31,5263,555,1,1,True,1,2015,7,31,31,c,a,1270,9,2008,0,1,1900,NaN,HE,23,16,8,9,6,3,98,54,18,1021,1018,1015,31.0,15.0,10.0,24,11,NaN,0.0,1,Fog,13,85,2008-09-15,2510
1,2,5,2015-07-31,6064,625,1,1,True,1,2015,7,31,31,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",TH,19,13,7,9,6,3,100,62,25,1021,1019,1017,10.0,10.0,10.0,14,11,NaN,0.0,4,Fog,309,80,2007-11-15,2815
2,3,5,2015-07-31,8314,821,1,1,True,1,2015,7,31,31,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",NW,21,13,6,10,7,4,100,61,24,1022,1019,1017,31.0,14.0,10.0,14,5,NaN,0.0,2,Fog,354,86,2006-12-15,3150
3,4,5,2015-07-31,13995,1498,1,1,True,1,2015,7,31,31,c,c,620,9,2009,0,1,1900,NaN,BE,19,14,9,9,7,4,94,61,30,1019,1017,1014,10.0,10.0,10.0,23,16,NaN,0.0,6,NaN,282,74,2009-09-15,2145
4,5,5,2015-07-31,4822,559,1,1,True,1,2015,7,31,31,a,a,29910,4,2015,0,1,1900,NaN,SN,20,15,10,8,6,5,82,55,26,1020,1018,1016,10.0,10.0,10.0,14,11,NaN,0.0,4,NaN,290,82,2015-04-15,107


In [264]:
display(DataFrameSummary(train_processed).summary())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,Week,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,trend,CompetitionOpenSince,CompetitionDaysOpen
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,992564,992564,992564,1.01721e+06,1.01721e+06,236167,1.01721e+06,936890,NaN,1.01721e+06,1.01721e+06,NaN,1.01721e+06
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647,2013.83,5.84676,15.7028,23.6155,NaN,NaN,5415.98,5.24476,1974.14,0.500564,12.1471,1955.94,NaN,NaN,14.3283,10.0739,5.90028,8.21675,5.835,3.26325,93.2838,73.9522,49.9234,1018.64,1015.43,1012.43,23.5428,12.1274,7.28499,22.6729,11.8708,48.4593,0.790271,5.5211,NaN,177.063,63.9962,NaN,14516.1
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056,0.777396,3.3261,8.78764,14.4334,NaN,NaN,7710.25,3.92852,50.8532,0.5,14.9479,55.8888,NaN,NaN,8.46478,7.23908,6.45973,6.14011,5.99337,6.03308,7.71733,13.2064,19.6264,7.91511,8.24871,8.74837,9.22237,4.87102,4.7714,8.94042,5.89719,13.1421,2.50262,1.68379,NaN,101.748,14.5175,NaN,18663.1
min,1,1,NaN,0,0,0,0,NaN,0,2013,1,1,1,NaN,NaN,0,1,1900,0,1,1900,NaN,NaN,-11,-13,-15,-14,-15,-73,44,30,4,976,974,970,0,0,0,3,2,21,0,0,NaN,-1,0,NaN,-956
25%,280,2,NaN,3727,405,1,0,NaN,0,2013,3,8,11,NaN,NaN,700,1,1900,0,1,1900,NaN,NaN,8,4,1,3,1,-1,88,64,34,1014,1011,1007,11,10,3,16,8,39,0,5,NaN,79,57,NaN,889
50%,558,4,NaN,5744,609,1,0,NaN,0,2014,6,16,22,NaN,NaN,2320,4,2006,1,1,2009,NaN,NaN,15,10,6,8,6,3,94,75,49,1019,1016,1013,31,11,8,21,11,47,0,6,NaN,201,65,NaN,2771
75%,838,6,NaN,7856,837,1,1,NaN,0,2014,8,23,35,NaN,NaN,6880,9,2011,1,22,2012,NaN,NaN,21,16,11,13,11,8,100,84,65,1024,1021,1018,31,14,10,27,14,55,0.25,7,NaN,257,73,NaN,41455
max,1115,7,NaN,41551,7388,1,1,NaN,1,2015,12,31,52,NaN,NaN,75860,12,2015,1,50,2015,NaN,NaN,38,31,24,25,20,18,100,100,100,1043,1040,1038,31,31,31,101,53,111,58.93,8,NaN,360,100,NaN,42200
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,509178,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,992564,992564,992564,1017209,1017209,236167,1017209,936890,774961,1017209,1017209,1017209,1017209
uniques,1115,7,942,21734,4086,2,2,2,2,3,12,31,52,4,3,655,12,23,2,24,8,3,12,50,45,40,39,36,38,52,71,93,66,67,68,24,32,24,42,27,45,40,9,21,362,68,171,11345


In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

class DataMapper(object):
    def __init__(self, data, max_cat_unique=10, drop=None, categorical=None, numeric=None):
        self._data = data
        self.type_map = self._get_map(max_cat_unique, drop, categorical, numeric)
        self.cat_map, self.num_map = None, None

    def _fill_cat_nan(self, data):
        data = data.copy()
        for col, col_type in self.type_map:
            if data[col].isnull().sum() > 0:
                if col_type == 'categorical':
                    data.loc[data[col].isnull(), col] = ''  # fill empty for categories NaN
        return data
        
    def fit(self, data=None):
        if data is None:
            data = self._data
        data = self._fill_cat_nan(data)
        missing_numeric = []
        for col, col_type in self.type_map:
            if col_type != 'categorical' and data[col].isnull().sum() > 0:
                missing_numeric.append(col)
        
        if len(missing_numeric) > 0:
            raise Exception('[' + ', '.join(missing_numeric) + '] has missing values')
        
        cat_map = DataFrameMapper([(col, LabelEncoder()) for col, col_type in self.type_map if col_type == 'categorical'])
        num_map = DataFrameMapper([(col, StandardScaler()) for col, col_type in self.type_map if col_type == 'numeric'])
        self.cat_map = cat_map.fit(data)
        self.num_map = num_map.fit(data)

    def apply(self, data):
        return self.cat_map.transform(data), self.num_map.transform(data)

    def _get_map(self, max_cat_unique=10, drop=None, categorical=None, numeric=None):
        null_info = []
        summary = DataFrameSummary(self._data)
        #type_maps = summary.columns_stats.copy().drop('missing_perc')
        type_maps = []
        
        for col in self._data:
            data_type = summary.columns_stats[col]['types']
#             if (drop and col in drop) or data_type not in [summary.TYPE_CATEGORICAL, summary.TYPE_BOOL,
#                                                            summary.TYPE_NUMERIC]:
#                 type_maps.drop(columns=[col], inplace=True)
#                 continue
            if drop and col in drop:
                continue
            map_type = ''
            if categorical and col in categorical:
                map_type = 'categorical'
            elif numeric and col in numeric:
                map_type = 'numeric'
            else:
                if data_type in [summary.TYPE_BOOL, summary.TYPE_CATEGORICAL]:
                    map_type = 'categorical'
                elif data_type == summary.TYPE_NUMERIC:
                    data_uniques = summary.columns_stats[col]['uniques']
                    if data_uniques <= max_cat_unique:
                        map_type = 'categorical'
                    else:
                        map_type = 'numeric'
            if summary.columns_stats[col]['missing'] > 0:
                if map_type == 'categorical':
                    null_info.append((col, ''))
                else:
                    null_info.append((col, 0))
            #type_maps[col]['types'] = map_type
            type_maps.append((col, map_type))
        return type_maps
mapper = DataMapper(train_processed, max_cat_unique=30,
                         categorical=['Store'],
                         drop=['Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM', 'Max_Gust_SpeedKm_h'])
mapper.fit()
train_mapped = mapper.apply(train_processed)

ValueError: Sales: Expected 2D array, got 1D array instead:
array=[5263. 6064. 8314. ...    0.    0.    0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

> /Users/kevin/miniconda3/envs/torch/lib/python3.6/site-packages/sklearn/utils/validation.py(441)check_array()
    439                     "Reshape your data either using array.reshape(-1, 1) if "
    440                     "your data has a single feature or array.reshape(1, -1) "
--> 441                     "if it contains a single sample.".format(array))
    442             array = np.atleast_2d(array)
    443             # To ensure that array flags are maintained

ipdb> arry.shape
*** NameError: name 'arry' is not defined
ipdb> array.shape
(1017209,)
ipdb> up
> /Users/kevin/miniconda3/envs/torch/lib/python3.6/site-packages/sklearn/preprocessing/data.py(612)partial_fit()
    610         """
    611         X = check_array(X, accept_sparse=('csr', 'csc'), copy=self.copy,
--> 612                         warn_on_dtype=True, estimator=self, dtype=FLOAT_DTYPES)
    613 
    614         # Even in the case of `with_mean=False`, we update the mean anyway

ipdb> X.shape
(1017209,)
ipdb> up


In [267]:

train_mapped = mapper.apply(train_processed)

TypeError: 'NoneType' object is not iterable

> /Users/kevin/miniconda3/envs/torch/lib/python3.6/site-packages/sklearn_pandas/dataframe_mapper.py(271)transform()
    269         extracted = []
    270         self.transformed_names_ = []
--> 271         for columns, transformers, options in self.built_features:
    272             input_df = options.get('input_df', self.input_df)
    273             # columns could be a string or list of

ipdb> q


In [258]:
train_processed.State

0

In [254]:
x.nonzero().count()

TypeError: count() takes exactly one argument (0 given)

> <ipython-input-254-40334334e394>(1)<module>()
----> 1 x.nonzero().count()

ipdb> 


In [247]:
mapper.fit()

Exception: CompetitionDistance, trend has missing values

> <ipython-input-246-295f49ca1a72>(24)fit()
     22 
     23         if len(missing_numeric) > 0:
---> 24             raise Exception(', '.join(missing_numeric) + ' has missing values')
     25 
     26         self.cat_map = DataFrameMapper([(col, LabelEncoder()) for col, col_type in self.type_map if col_type == 'categorical'])

ipdb> q


In [244]:
mapper.type_map.loc['missing'] > 0 and mapper.type_map.loc['missing'] > 0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

> /Users/kevin/miniconda3/envs/torch/lib/python3.6/site-packages/pandas/core/generic.py(1121)__nonzero__()
   1119         raise ValueError("The truth value of a {0} is ambiguous. "
   1120                          "Use a.empty, a.bool(), a.item(), a.any() or a.all()."
-> 1121                          .format(self.__class__.__name__))
   1122 
   1123     __bool__ = __nonzero__

ipdb> q


In [243]:
mapper.type_map[mapper.type_map.columns[mapper.type_map.loc['missing'] > 0]]

,CompetitionDistance,PromoInterval,CloudCover,Events,trend
counts,1014567,509178,936890,774961,996485
uniques,654,3,9,21,67
missing,2642,508031,80319,242248,20724
types,numeric,categorical,categorical,categorical,numeric


In [241]:
mapper.type_map[mapper.type_map.loc['missing'][mapper.type_map.loc['missing'] > 0]]

KeyError: '[2642 508031 80319 242248 20724] not in index'

> /Users/kevin/miniconda3/envs/torch/lib/python3.6/site-packages/pandas/core/indexing.py(1269)_convert_to_indexer()
   1267                 if mask.any():
   1268                     raise KeyError('{mask} not in index'
-> 1269                                    .format(mask=objarr[mask]))
   1270 
   1271                 return _values_from_object(indexer)

ipdb> q


In [221]:
summary.columns_stats.drop()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,Week,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,trend,CompetitionOpenSince
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1014567,1017209,1017209,1017209,1017209,1017209,509178,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,992564,992564,992564,1017209,1017209,236167,1017209,936890,774961,1017209,996485,1017209
uniques,1115,7,942,21734,4086,2,2,2,2,3,12,31,52,4,3,654,12,23,2,24,8,3,12,50,45,40,39,36,38,52,71,93,66,67,68,24,32,24,42,27,45,40,9,21,362,67,171
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2642,0,0,0,0,0,508031,0,0,0,0,0,0,0,0,0,0,0,0,0,24645,24645,24645,0,0,781042,0,80319,242248,0,20724,0
types,numeric,numeric,date,numeric,numeric,bool,bool,bool,bool,numeric,numeric,numeric,numeric,categorical,categorical,numeric,numeric,numeric,bool,numeric,numeric,categorical,categorical,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,categorical,numeric,numeric,date


In [211]:
for col, t in train_map_info:
    print(t, train_processed[train_processed[col].isnull()][col])

numeric Series([], Name: Store, dtype: int64)
categorical Series([], Name: DayOfWeek, dtype: int64)
numeric Series([], Name: Sales, dtype: int64)
numeric Series([], Name: Customers, dtype: int64)
categorical Series([], Name: Open, dtype: int64)
categorical Series([], Name: Promo, dtype: int64)
categorical Series([], Name: StateHoliday, dtype: bool)
categorical Series([], Name: SchoolHoliday, dtype: int64)
categorical Series([], Name: Year, dtype: int64)
categorical Series([], Name: Month, dtype: int64)
numeric Series([], Name: Day, dtype: int64)
numeric Series([], Name: Week, dtype: int64)
categorical Series([], Name: StoreType, dtype: object)
categorical Series([], Name: Assortment, dtype: object)
numeric 290       NaN
621       NaN
878       NaN
1405      NaN
1736      NaN
1993      NaN
2520      NaN
2851      NaN
3108      NaN
3635      NaN
3966      NaN
4223      NaN
4750      NaN
5081      NaN
5338      NaN
5865      NaN
6196      NaN
6453      NaN
6980      NaN
7311      NaN
7568